Setup loaders...

In [8]:
import numpy as np
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("data/lfw/*/*"))
dog_files = np.array(glob("data/dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

There are 13233 total human images.
There are 8351 total dog images.


In [1]:
# batch loader...

import os
import torch
from torchvision import datasets,transforms
from PIL import Image
import matplotlib.pyplot as plt

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
# variables
batch_size = 10
num_workers = 1

# image paths
train_folder = "data/dogImages/train"
valid_folder = "data/dogImages/valid"
test_folder = "data/dogImages/test"

# transforms
train_transforms = transforms.Compose([transforms.ColorJitter(brightness=0.25,saturation=0.15),
                                    transforms.RandomRotation(15),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomResizedCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])]) 
test_transforms = transforms.Compose([transforms.Resize(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])]) 


# data loaders
train_data = datasets.ImageFolder(train_folder, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_folder, transform=test_transforms)
test_data = datasets.ImageFolder(test_folder, transform=test_transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

# loaders dict
loaders_scratch={
    "train":train_loader,
    "valid":valid_loader,
    "test": test_loader
}

print('Number of training images: ' + str(len(train_data)))
print('Number of validation images: ' + str(len(valid_data)))
print('Number of testing images: ' + str(len(test_data)))

print('Transforms and Loaders setup...')


Number of training images: 6680
Number of validation images: 835
Number of testing images: 836
Transforms and Loaders setup...


CNN Simple Example

In [18]:
import torch
import torchvision.models as models

# define VGG16 model
pre_model = models.resnet18(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    pre_model = pre_model.cuda()
    
print('Pretrained resnet18 model setup... ')

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/emaggard/.torch/models/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:19<00:00, 2383641.02it/s]

Pretrained resnet18 model setup... 


In [4]:
from PIL import Image
import torchvision.transforms as transforms

# Set PIL to be tolerant of image files that are truncated.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def pre_predict(img_path):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to 
    predicted ImageNet class for image at specified path
    
    Args:
        img_path: path to an image
        
    Returns:
        Index corresponding to VGG-16 model's prediction
    '''
    
    ## TODO: Complete the function.
    ## Load and pre-process an image from the given img_path
    pre_model.eval()  # need to set to eval mode
    img = Image.open(img_path)
    img_transform = transforms.Compose([
                        transforms.Resize(size=(224, 224)),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])

    img_trans = img_transform(img).unsqueeze(0)
    
    ## Return the *index* of the predicted class for that image
    if use_cuda:
        img_trans = img_trans.cuda()
        
    pred = pre_model(img_trans)
    top_class = pred.topk(1)[1].item()
    
    return top_class # predicted class index

In [19]:
from tqdm import tqdm

human_files_short = human_files[:100]
dog_files_short = dog_files[:100]

### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
dog_true = 0;
for img in human_files_short:
    if dog_detector(img):
        dog_true += 1;
        
print('Number of dogs detected: ' + str(dog_true) + ' out of ' + str(len(human_files_short)) + ' humans.  Percentage: ' + str((dog_true/len(human_files_short))*100))

dog_true = 0;
for img in dog_files_short:
    if dog_detector(img):
        dog_true += 1;
        
print('Number of dogs detected: ' + str(dog_true) + ' out of ' + str(len(dog_files_short)) + ' dogs.  Percentage: ' + str((dog_true/len(dog_files_short))*100))


Number of dogs detected: 0 out of 100 humans.  Percentage: 0.0
Number of dogs detected: 94 out of 100 dogs.  Percentage: 94.0


VGG16 Example

In [3]:
import torch
import torchvision.models as models

# define VGG16 model
pre_model = models.vgg16(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    pre_model = pre_model.cuda()
    
print('Pretrained VGG16 model setup... ')

Pretrained VGG16 model setup... 


In [5]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    ## TODO: Complete the function.
    pred_class = pre_predict(img_path)
    return pred_class >= 151 and pred_class <= 268 # true/false

In [9]:
from tqdm import tqdm

human_files_short = human_files[:100]
dog_files_short = dog_files[:100]

### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
dog_true = 0;
for img in human_files_short:
    if dog_detector(img):
        dog_true += 1;
        
print('Number of dogs detected: ' + str(dog_true) + ' out of ' + str(len(human_files_short)) + ' humans.  Percentage: ' + str((dog_true/len(human_files_short))*100))

dog_true = 0;
for img in dog_files_short:
    if dog_detector(img):
        dog_true += 1;
        
print('Number of dogs detected: ' + str(dog_true) + ' out of ' + str(len(dog_files_short)) + ' dogs.  Percentage: ' + str((dog_true/len(dog_files_short))*100))


Number of dogs detected: 0 out of 100 humans.  Percentage: 0.0
Number of dogs detected: 93 out of 100 dogs.  Percentage: 93.0


ResNet Example

In [10]:
import torch
import torchvision.models as models

# define ResNet model
pre_model = models.resnet50(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    pre_model = pre_model.cuda()

In [11]:
from tqdm import tqdm

human_files_short = human_files[:100]
dog_files_short = dog_files[:100]

### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
dog_true = 0;
for img in human_files_short:
    if dog_detector(img):
        dog_true += 1;
        
print('Number of dogs detected: ' + str(dog_true) + ' out of ' + str(len(human_files_short)) + ' humans.  Percentage: ' + str((dog_true/len(human_files_short))*100))

dog_true = 0;
for img in dog_files_short:
    if dog_detector(img):
        dog_true += 1;
        
print('Number of dogs detected: ' + str(dog_true) + ' out of ' + str(len(dog_files_short)) + ' dogs.  Percentage: ' + str((dog_true/len(dog_files_short))*100))


Number of dogs detected: 1 out of 100 humans.  Percentage: 1.0
Number of dogs detected: 96 out of 100 dogs.  Percentage: 96.0


Alexnet Example

In [14]:
import torch
import torchvision.models as models

# define ResNet model
pre_model = models.alexnet(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    pre_model = pre_model.cuda()

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/emaggard/.torch/models/alexnet-owt-4df8aa71.pth
100%|██████████| 244418560/244418560 [01:55<00:00, 2122888.56it/s]


In [15]:
### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
dog_true = 0;
for img in human_files_short:
    if dog_detector(img):
        dog_true += 1;
        
print('Number of dogs detected: ' + str(dog_true) + ' out of ' + str(len(human_files_short)) + ' humans.  Percentage: ' + str((dog_true/len(human_files_short))*100))

dog_true = 0;
for img in dog_files_short:
    if dog_detector(img):
        dog_true += 1;
        
print('Number of dogs detected: ' + str(dog_true) + ' out of ' + str(len(dog_files_short)) + ' dogs.  Percentage: ' + str((dog_true/len(dog_files_short))*100))


Number of dogs detected: 1 out of 100 humans.  Percentage: 1.0
Number of dogs detected: 94 out of 100 dogs.  Percentage: 94.0


GoogleNet Example

In [17]:
import torch
import torchvision.models as models

# define ResNet model
pre_model = models.googlenet(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    pre_model = pre_model.cuda()

AttributeError: module 'torchvision.models' has no attribute 'googlenet'